# **Kaggle Project**
## **Irem Nesli Erez & Ezgi Siir Kibris**

In [1]:
# This section of the code is added by default.

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [62]:
# Import the necessary packages

import nltk #language processing
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

import re
import emoji

from sklearn.feature_extraction.text import CountVectorizer #for bag of words

import gensim
from gensim.models import Word2Vec



path = '/Users/nesli/Desktop/SPRING 2022/DSCC 465/Kaggle_istanbul/'
os.chdir(path)

import random
random.seed(465) #seed

## Read the data

In [3]:
# Training data set
data_train = pd.read_csv('congressional_tweet_training_data.csv')
data_train

,favorite_count,full_text,hashtags,retweet_count,year,party_id
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D
...,...,...,...,...,...,...
592798,3,"b'This time, it focused on careers in #publics...",publicservice publicsafety,0,2017.0,R
592799,5,"b'.#StormyDaniels, #MichaelWolfe, #JamesComey ...",StormyDaniels MichaelWolfe JamesComey,1,2018.0,R
592800,33,b'@NRDems The American people deserve the trut...,CultureOfCorruption,14,2020.0,D
592801,4,b'Only 2 weeks left to submit your #app to the...,app copolitics CAC16 HouseOfCode co06,3,2016.0,R


In [4]:
# Test data set
data_test = pd.read_csv('congressional_tweet_test_data.csv')
data_test

,Id,favorite_count,full_text,hashtags,retweet_count,year,party
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D
1,1,27,"b'This #NativeWomensEqualPay Day, we recommit ...",NativeWomensEqualPay,11,NaN,D
2,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D
3,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D
4,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D
...,...,...,...,...,...,...,...
264995,264995,516,b'We need to #ExtendCHIP before a single child...,ExtendCHIP,223,2017.0,D
264996,264996,0,"b""Our #ObamaCare investigation continued today...",ObamaCare,3,2013.0,D
264997,264997,1,b'Congratulations to the new #MissTeenUSA Loga...,MissTeenUSA CT,4,2012.0,D
264998,264998,2174,b'Speaking of dishonesty. Nothing like being c...,mosen,1168,2018.0,D


## Kodu hizlandirmak icin

In [5]:
data_train[0:100]

,favorite_count,full_text,hashtags,retweet_count,year,party_id
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D
...,...,...,...,...,...,...
95,35,"b'In 5 years of DACA, over 780,000 #DREAMers h...",DREAMers SaveDACA,17,2017.0,D
96,118,b'Over 90% of gun owners support expanding bac...,OutShoutTheGunLobby,39,2017.0,D
97,26,b'Holi is a time to celebrate renewal &amp; a ...,Holi2015,21,2015.0,D
98,78,b'Connecticut\xe2\x80\x99s 4th District lost t...,911remembrance,6,2018.0,D


In [6]:
data_train=data_train[0:100]

In [7]:
data_test=data_test[0:100]

## Data Pre-processing

First, preprocess the full_text feature of the tweets for both test and the train data sets.

In [8]:
#Code from HW4 CITE IT!

In [9]:
# Start by cleaning the full text of the tweets 
data_train.dropna(subset=['full_text'],inplace=True) #First, I drop the rows with NA in the text column
data_train

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,favorite_count,full_text,hashtags,retweet_count,year,party_id
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D
...,...,...,...,...,...,...
95,35,"b'In 5 years of DACA, over 780,000 #DREAMers h...",DREAMers SaveDACA,17,2017.0,D
96,118,b'Over 90% of gun owners support expanding bac...,OutShoutTheGunLobby,39,2017.0,D
97,26,b'Holi is a time to celebrate renewal &amp; a ...,Holi2015,21,2015.0,D
98,78,b'Connecticut\xe2\x80\x99s 4th District lost t...,911remembrance,6,2018.0,D


In [10]:
data_train=data_train.reset_index(drop=True) #to fix the indices

In [11]:
#Tokenize
tokens_list=list()
for i in np.arange(len(data_train)):
    tokens_list.append(nltk.word_tokenize(str(data_train['full_text'][i])))

In [12]:
tagged_tokens=list()
for i in np.arange(len(tokens_list)):
    tagged_tokens.append(nltk.pos_tag(tokens_list[i]))

In [13]:
def get_wordnet_pos(treebank_tag): #Following from [16]

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None # for easy if-statement 

In [14]:
lemmatizer = WordNetLemmatizer()

In [15]:
lemma_list=list()

for i in np.arange(len(tagged_tokens)):
    lemma=list()
    for j in np.arange(len(tagged_tokens[i])):
        
        token=tagged_tokens[i][j][0] 
        tag=tagged_tokens[i][j][1]
        
        wntag = get_wordnet_pos(tag)
        if wntag is None:           #do not supply tag in case of None
            lemma.append(lemmatizer.lemmatize(token))
        else:
            lemma.append(lemmatizer.lemmatize(token, pos=wntag))
    lemma_list.append(lemma)
        

In [16]:
len(lemma_list)

100

In [17]:
# Before removing elements from the string, make all of them lowercase
filtered_lower=list()
for i in np.arange(len(lemma_list)):
    lowered=list()
    for j in np.arange(len(lemma_list[i])):
        lowered.append(lemma_list[i][j].lower())
    filtered_lower.append(lowered)  

In [18]:
# Now to remove stop words like 'the', 'a', 'and' 
stop_words = set(stopwords.words('english'))

In [19]:
filtered_list=list()
for i in np.arange(len(filtered_lower)):
    filtered_list.append([w for w in filtered_lower[i] if not w.lower() in stop_words])

In [20]:
len(filtered_list)

100

In [21]:
# Remove numbers, words <2 characters, punctuation, links and emojis 

def emoji_free_text(text): # From [9]
    return emoji.get_emoji_regexp().sub(r'', text)

In [22]:
filtered_elements=list()

for i in np.arange(len(filtered_list)):
    filter_element=list()
    for j in np.arange(len(filtered_list[i])):
        
        element=re.sub(r'\d+', '',filtered_list[i][j])          # Remove numbers
        element=re.sub(r'\b\w{1}\b', '', element)               # Remove <2 characters
        element=re.sub(r'[^\w\s]', '', element)                 # Remove punctuation
        element=re.sub(r'http\S+', '', element)                 # Remove links
        #element=re.sub('/[\u{1f300}-\u{1f5ff}]/', '', element) # Remove symbols
        #element=re.sub('/[\u{1f600}-\u{1f64f}]/','', element)  # Remove emoticons
        emoji_free_text(element)
        
        filter_element.append(element)
    filtered_elements.append(filter_element)   

In [23]:
# Combine the elements back to form sentences
text_clean_list=list()
s=' '

for i in np.arange(len(filtered_elements)):
    text_clean_list.append(s.join(filtered_elements[i]))

In [24]:
len(text_clean_list)

100

In [25]:
# Add a new column of 'text_clean'
data_train.insert(6, "text_clean", text_clean_list)

In [26]:
data_train.head()

,favorite_count,full_text,hashtags,retweet_count,year,party_id,text_clean
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,rt kusinews one longtime viewer congressma...
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,today urge cdcgov immediately launch phon...
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,tomorrow mo senior graduate calvary lutheran...
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,congrats teamusa canton native jgreenway win...
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,pleased support amergateways june fiesta hon...


In [27]:
# Repeat the same for the test data

# Start by cleaning the full text of the tweets 
data_test.dropna(subset=['full_text'],inplace=True) #First, I drop the rows with NA in the text column
data_test

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Id,favorite_count,full_text,hashtags,retweet_count,year,party
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D
1,1,27,"b'This #NativeWomensEqualPay Day, we recommit ...",NativeWomensEqualPay,11,NaN,D
2,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D
3,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D
4,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D
...,...,...,...,...,...,...,...
95,95,66,"b""I've never been more confused in my life. An...",whiteandgold blackandblue,78,2015.0,D
96,96,678,b'This is progress #coronarvirus https://t.co/...,coronarvirus,224,2020.0,D
97,97,5,b'.@HouseGOP voted to reduce child care access...,WEmatter,11,2014.0,D
98,98,0,b'Our Army is equally ready to assist in a dis...,MilitaryMonday,0,2016.0,D


In [28]:
data_test=data_test.reset_index(drop=True) #to fix the indices

In [29]:
#Tokenize
tokens_list=list()
for i in np.arange(len(data_test)):
    tokens_list.append(nltk.word_tokenize(str(data_test['full_text'][i])))

In [30]:
tagged_tokens=list()
for i in np.arange(len(tokens_list)):
    tagged_tokens.append(nltk.pos_tag(tokens_list[i]))

In [31]:
def get_wordnet_pos(treebank_tag): #Following from [16]

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None # for easy if-statement 

In [32]:
lemmatizer = WordNetLemmatizer()

In [33]:
lemma_list=list()

for i in np.arange(len(tagged_tokens)):
    lemma=list()
    for j in np.arange(len(tagged_tokens[i])):
        
        token=tagged_tokens[i][j][0] 
        tag=tagged_tokens[i][j][1]
        
        wntag = get_wordnet_pos(tag)
        if wntag is None:           #do not supply tag in case of None
            lemma.append(lemmatizer.lemmatize(token))
        else:
            lemma.append(lemmatizer.lemmatize(token, pos=wntag))
    lemma_list.append(lemma)
        

In [34]:
len(lemma_list)

100

In [35]:
# Before removing elements from the string, make all of them lowercase
filtered_lower=list()
for i in np.arange(len(lemma_list)):
    lowered=list()
    for j in np.arange(len(lemma_list[i])):
        lowered.append(lemma_list[i][j].lower())
    filtered_lower.append(lowered)  

In [36]:
# Now to remove stop words like 'the', 'a', 'and' 
stop_words = set(stopwords.words('english'))

In [37]:
filtered_list=list()
for i in np.arange(len(filtered_lower)):
    filtered_list.append([w for w in filtered_lower[i] if not w.lower() in stop_words])

In [38]:
len(filtered_list)

100

In [39]:
# Remove numbers, words <2 characters, punctuation, links and emojis 

def emoji_free_text(text): # From [9]
    return emoji.get_emoji_regexp().sub(r'', text)

In [40]:
filtered_elements=list()

for i in np.arange(len(filtered_list)):
    filter_element=list()
    for j in np.arange(len(filtered_list[i])):
        
        element=re.sub(r'\d+', '',filtered_list[i][j])          # Remove numbers
        element=re.sub(r'\b\w{1}\b', '', element)               # Remove <2 characters
        element=re.sub(r'[^\w\s]', '', element)                 # Remove punctuation
        element=re.sub(r'http\S+', '', element)                 # Remove links
        #element=re.sub('/[\u{1f300}-\u{1f5ff}]/', '', element) # Remove symbols
        #element=re.sub('/[\u{1f600}-\u{1f64f}]/','', element)  # Remove emoticons
        emoji_free_text(element)
        
        filter_element.append(element)
    filtered_elements.append(filter_element)   

In [41]:
# Combine the elements back to form sentences
text_clean_list=list()
s=' '

for i in np.arange(len(filtered_elements)):
    text_clean_list.append(s.join(filtered_elements[i]))

In [42]:
len(text_clean_list)

100

In [43]:
# Add a new column of 'text_clean'
data_test.insert(7, "text_clean", text_clean_list)

In [44]:
data_test.head()

,Id,favorite_count,full_text,hashtags,retweet_count,year,party,text_clean
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D,taxreform improve playing field american wo...
1,1,27,"b'This #NativeWomensEqualPay Day, we recommit ...",NativeWomensEqualPay,11,NaN,D,this nativewomensequalpay day recommit pass ...
2,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D,xexci become convinced generation silence ma...
3,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D,during nationaladoptionmonth honor adoptive ...
4,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D,happy airborneday usarmy paratrooper veteran...


## How to deal with shortened words like rt?

We could create a dictionary for that, but we assume that the senators(?) use English properly.

## Vectorize the data

Now for vectorization, we can use different techniques. Try two different methods:

1. Bag of words
2. Word2vec

We will not try TF-IDF because it is rather beneficial for long text data.

Considering the fact that the bag of words method includes each unique word in the text as a dimension, it is better to apply vectorization on the merged data set that contains both the test and the training sets.

In [45]:
# Bag of words
# Following from [1]
vectorizer = CountVectorizer()

text_bow = vectorizer.fit_transform(data_train['text_clean'])
text_bow_df = pd.DataFrame(text_bow.toarray(),columns=vectorizer.get_feature_names())

In [46]:
text_bow_df

,abrams,aca,access,accessibility,across,act,action,activity,actonclimate,ada,...,xexcmy,xexdxaxefxbxf,xexm,yarnell,year,yorkers,young,youtube,zika,zumwalt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
96,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# Bag of words
# Following from [1]
# For the test data
vectorizer = CountVectorizer()

text_bow_test = vectorizer.fit_transform(data_test['text_clean'])
text_bow__test_df = pd.DataFrame(text_bow_test.toarray(),columns=vectorizer.get_feature_names())

In [48]:
text_bow__test_df

,abcnews,able,abortion,absolutely,access,accomplish,accordingly,across,act,actxexd,...,xfxfxaxa,xfxfxbb,year,yemen,yesterday,yet,young,youth,yr,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


But we would like the same vectorization to happen for all of the data.

In [49]:
data=pd.concat([data_train,data_test], ignore_index=True)

In [50]:
data

,favorite_count,full_text,hashtags,retweet_count,year,party_id,text_clean,Id,party
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,rt kusinews one longtime viewer congressma...,NaN,NaN
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,today urge cdcgov immediately launch phon...,NaN,NaN
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,tomorrow mo senior graduate calvary lutheran...,NaN,NaN
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,congrats teamusa canton native jgreenway win...,NaN,NaN
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,pleased support amergateways june fiesta hon...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
195,66,"b""I've never been more confused in my life. An...",whiteandgold blackandblue,78,2015.0,NaN,ve never confused life ve listen replouieg...,95.0,D
196,678,b'This is progress #coronarvirus https://t.co/...,coronarvirus,224,2020.0,NaN,this progress coronarvirus http cohsplcjy,96.0,D
197,5,b'.@HouseGOP voted to reduce child care access...,WEmatter,11,2014.0,NaN,housegop vote reduce child care access de...,97.0,D
198,0,b'Our Army is equally ready to assist in a dis...,MilitaryMonday,0,2016.0,NaN,our army equally ready assist disaster amp e...,98.0,D


In [51]:
# Bag of words
# Following from [1]
vectorizer = CountVectorizer()

text_bow = vectorizer.fit_transform(data['text_clean'])
text_bow_df = pd.DataFrame(text_bow.toarray(),columns=vectorizer.get_feature_names())

In [52]:
text_bow_df

,abcnews,able,abortion,abrams,absolutely,aca,access,accessibility,accomplish,accordingly,...,yesterday,yet,yorkers,young,youth,youtube,yr,zero,zika,zumwalt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
text_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 4],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [54]:
#Now, include these vectors as a new column named bow_vector in your datasets 

data_train['bow_vector']=text_bow[0:len(data_train)].toarray().tolist()

In [55]:
data_train

,favorite_count,full_text,hashtags,retweet_count,year,party_id,text_clean,bow_vector
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,rt kusinews one longtime viewer congressma...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,today urge cdcgov immediately launch phon...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,tomorrow mo senior graduate calvary lutheran...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,congrats teamusa canton native jgreenway win...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,pleased support amergateways june fiesta hon...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
95,35,"b'In 5 years of DACA, over 780,000 #DREAMers h...",DREAMers SaveDACA,17,2017.0,D,in year daca dreamers give chance work stu...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
96,118,b'Over 90% of gun owners support expanding bac...,OutShoutTheGunLobby,39,2017.0,D,over gun owner support expand background che...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
97,26,b'Holi is a time to celebrate renewal &amp; a ...,Holi2015,21,2015.0,D,holi time celebrate renewal amp strengthenin...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
98,78,b'Connecticut\xe2\x80\x99s 4th District lost t...,911remembrance,6,2018.0,D,connecticutxexs th district lose many mom dad...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [56]:
#Now, include these vectors as a new column named bow_vector in your datasets 

data_test['bow_vector']=text_bow[len(data_train):len(data)].toarray().tolist()

In [57]:
data_test

,Id,favorite_count,full_text,hashtags,retweet_count,year,party,text_clean,bow_vector
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D,taxreform improve playing field american wo...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,27,"b'This #NativeWomensEqualPay Day, we recommit ...",NativeWomensEqualPay,11,NaN,D,this nativewomensequalpay day recommit pass ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
2,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D,xexci become convinced generation silence ma...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D,during nationaladoptionmonth honor adoptive ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D,happy airborneday usarmy paratrooper veteran...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
95,95,66,"b""I've never been more confused in my life. An...",whiteandgold blackandblue,78,2015.0,D,ve never confused life ve listen replouieg...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
96,96,678,b'This is progress #coronarvirus https://t.co/...,coronarvirus,224,2020.0,D,this progress coronarvirus http cohsplcjy,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
97,97,5,b'.@HouseGOP voted to reduce child care access...,WEmatter,11,2014.0,D,housegop vote reduce child care access de...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
98,98,0,b'Our Army is equally ready to assist in a dis...,MilitaryMonday,0,2016.0,D,our army equally ready assist disaster amp e...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [67]:
# Now Word2vec

# Following from [2]

model = gensim.models.Word2Vec(data, min_count = 1)

In [68]:
model

In [ ]:
# + T

## **References**

[1] https://www.analyticsvidhya.com/blog/2021/08/a-friendly-guide-to-nlp-bag-of-words-with-python-example/

[2] https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/